# veloVI benchmark on cell cycle data

Notebook benchmarks velocity, latent time and GRN inference, and cross boundary correctness using RegVelo on cell cycle data.

## Library imports

In [9]:
import pandas as pd

import anndata as ad
import scanpy as sc
import scvelo as scv
import scvi
from velovi import VELOVI

from rgv_tools import DATA_DIR
from rgv_tools.benchmarking import set_output

## General settings

In [2]:
scv.settings.verbosity = 3

## Constants

In [3]:
DATASET = "cell_cycle"

In [4]:
STATE_TRANSITIONS = [("G1", "S"), ("S", "G2M")]

In [5]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / "results").mkdir(parents=True, exist_ok=True)

In [6]:
nn_levels = [10, 30, 50, 70, 90, 100]

## Functions

In [7]:
def compute_confidence(adata, vkey="velocity"):
    """
    Compute the velocity confidence for a given AnnData object.

    Parameters:
    -----------
    adata : AnnData
        Annotated data matrix (single-cell data) containing layers for velocity analysis.
    vkey : str, optional (default="velocity")
        Key in adata.layers corresponding to the velocity matrix.

    Returns:
    --------
    g_df : pandas.DataFrame
        DataFrame containing the velocity confidence scores for each cell.
    """
    adata.layers[vkey]
    scv.tl.velocity_graph(adata, vkey=vkey, n_jobs=1)
    scv.tl.velocity_confidence(adata, vkey=vkey)

    g_df = pd.DataFrame()
    g_df[f"{vkey}_consistency"] = adata.obs[f"{vkey}_confidence"].to_numpy().ravel()

    return g_df

## Velocity pipeline

In [10]:
for level in nn_levels:
    adata = ad.io.read_h5ad(DATA_DIR / "processed" / f"adata_processed_nn{level}.h5ad")
    sc.pp.neighbors(
        adata, n_neighbors=30, n_pcs=30
    )  # redefine neighborhood graph to ensure graph is the same when calculate velocity consistency

    print(adata)
    scvi.settings.seed = 0
    VELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")
    vae = VELOVI(adata)
    vae.train(max_epochs=1500)

    set_output(adata, vae, n_samples=30)

    t_df = compute_confidence(adata, vkey="fit_t")
    t_df["Dataset"] = "Cell cycle"
    t_df["Method"] = "velovi"

    v_df = compute_confidence(adata, vkey="velocity")
    v_df["Dataset"] = "Cell cycle"
    v_df["Method"] = "velovi"

    t_df.to_parquet(path=DATA_DIR / "results" / f"velovi_confidence_time_{level}.parquet")

    v_df.to_parquet(path=DATA_DIR / "results" / f"velovi_confidence_velocity_{level}.parquet")

2025-09-29 19:45:37.533643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759167938.389113 1655430 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759167938.813134 1655430 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[rank: 0] Seed set to 0


AnnData object with n_obs × n_vars = 1146 × 354
    obs: 'phase', 'fucci_time', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'ensum_id', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'log1p', 'neighbors', 'pca', 'velocity_params'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'Ms', 'Mu', 'spliced', 'total', 'unspliced', 'velocity'
    obsp: 'connectivities', 'distances'


/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command 

Training:   0%|          | 0/1500 [00:00<?, ?it/s]

Monitored metric elbo_validation did not improve in the last 45 records. Best score: -831.898. Signaling Trainer to stop.
computing velocity graph (using 1/112 cores)


...

Monitored metric elbo_validation did not improve in the last 45 records. Best score: -495.670. Signaling Trainer to stop.
computing velocity graph (using 1/112 cores)


  0%|          | 0/1146 [00:00<?, ?cells/s]

    finished (0:00:01) --> added 
    'fit_t_graph', sparse matrix with cosine correlations (adata.uns)
--> added 'fit_t_length' (adata.obs)
--> added 'fit_t_confidence' (adata.obs)
--> added 'fit_t_confidence_transition' (adata.obs)
computing velocity graph (using 1/112 cores)


  0%|          | 0/1146 [00:00<?, ?cells/s]

    finished (0:00:01) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)


In [12]:
v_df

,velocity_consistency,Dataset,Method
0,0.759060,Cell cycle,velovi
1,0.762383,Cell cycle,velovi
2,0.780014,Cell cycle,velovi
3,0.585037,Cell cycle,velovi
4,0.895268,Cell cycle,velovi
...,...,...,...
1141,0.849652,Cell cycle,velovi
1142,0.795326,Cell cycle,velovi
1143,0.827614,Cell cycle,velovi
1144,0.838542,Cell cycle,velovi
